In [175]:
import pickle
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score 

In [139]:
# Acquiring the tabnet information
def downloading_class_probabilities(model_name):
    path = '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/pickles/'
    with open(path+ model_name +'_pickles/class_probs.pickle', 'r+b') as handle:
        b = pickle.load(handle)
    return b

def downloading_class_predictions(model_name):
    path = '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/pickles/'
    with open(path+ model_name +'_pickles/predictions.pickle', 'r+b') as handle:
        b = pickle.load(handle)
    return b

def downloading_tabnet_val(model_name):
    path = '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/pickles/val_order_pickles'
    with open(path+ '/' + model_name +'.pickle', 'r+b') as handle:
        b = pickle.load(handle)
    return b

"Optional model_names:\n'tabnet'\n'1d_cnn\n'simpleNN'\n"

In [150]:
def check_that_labels_are_identical(tabnet_labels, simpleNN_labels, CNN_labels):
    simpleNN = torch.argmax(simpleNN_labels, 1).numpy()
    CNN = torch.argmax(CNN_labels, 1).numpy()
    tabnet = pd.to_numeric(tabnet_labels).to_numpy()
    assert tabnet.all() == simpleNN.all() == CNN.all()

# Psuedocode
1. Check that the label values are identical for tabnet, simpleNN, 1DCNN.
2. Load all of the class probabilities
3. multiply and add according to weighting scheme
4. compare ensemble method to prediction

# 1

In [148]:
# Load the validation pickles
l = torch.load('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/pickles/val_order_pickles/simpleNN_val')
w = torch.load('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/pickles/val_order_pickles/1DCNN_val')
tabnet_labels = downloading_tabnet_val("tab_val")

In [151]:
# perform check on validation
check_that_labels_are_identical(tabnet_labels, l, w)

# 2 

In [24]:
# load class probs for all three models
tabnet_class_probs = downloading_class_probabilities('tabnet')

the numpy array is the correct array

In [25]:
tabnet_class_preds = downloading_class_predictions('tabnet')

functional!

In [153]:
x = '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/Best_Tomics_Model/saved_models/Tomics_SimpleNN'
state_SimpleNN = torch.load(x)

In [168]:
NN_pp = state_SimpleNN['predict_proba'].cpu().numpy()

In [155]:
x = '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/Best_Tomics_Model/saved_models/Tomics_1DCNN'
state_1DCNN = torch.load(x)

In [156]:
state['predictions']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [167]:
CC_pp = state_1DCNN['predict_proba'].cpu().numpy()

# 3

In [219]:
combined = CC_pp * 0.2 + NN_pp * 0.7 + tabnet_class_probs*0.1


In [220]:
combined_pred = np.argmax(combined, axis = 1)

# 4

In [221]:
f1_score(combined_pred, pd.to_numeric(tabnet_labels).to_numpy(), average = 'macro')

0.44754130223517974

In [222]:
accuracy_score(combined_pred, pd.to_numeric(tabnet_labels).to_numpy())

0.6746411483253588

## Summary
not combination really made a difference.